# Дз 2. Spark Core API


### напоминание
лучший друг: https://spark.apache.org/docs/2.4.8/api/python/pyspark.html

скачаем и распакуем данные

In [239]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip -o smsspamcollection.zip
!pip install pandas

--2022-02-11 15:52:46--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip.1’

smsspamcollection.z 100%[===================>] 198.65K   270KB/s    in 0.7s    

2022-02-11 15:52:48 (270 KB/s) - ‘smsspamcollection.zip.1’ saved [203415/203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  
     |████████████████████████████████| 11.3 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 60.9 MB/s eta 0:00:01
You should consider upgrading via the '/jupyter/bin/python3.7 -m pip install --upgrade pip' command.


In [340]:
import pyspark
import pandas as pd
import collections
from collections import Counter

In [12]:
sc = pyspark.sql.SparkSession.builder.getOrCreate().sparkContext
src = sc.textFile("SMSSpamCollection")

Это формат tsv, две колонки разделенные табуляцией (`\t`). В первой метка класса — `spam` или `ham`, во второй — текст sms

In [13]:
src.take(5)

['ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'ham\tOk lar... Joking wif u oni...',
 "spam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'ham\tU dun say so early hor... U c already then say...',
 "ham\tNah I don't think he goes to usf, he lives around here though"]

### Задание 1. Читаем и парсим датасет.
Распарсите датасет так, чтобы он содержал только тексты sms. Должно получиться что-то такое:

```python
['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]
```

In [257]:
src.map(lambda s: s.split("\t")[1]).take(5)

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [52]:
type(src)

pyspark.rdd.RDD

In [260]:
all_text = src.map(lambda s: s.split("\t")[1]).collect()
type(all_text)

list

In [285]:
text = src.map(lambda s: s.split("\t")[1])

### Задание 2. Считаем баланс классов
Подсчитайте сколько в датасете текстов с метками spam и метками ham. Должны получиться такие числа:

```python
defaultdict(int, {'ham': 4827, 'spam': 747})
```

In [80]:
src.map(lambda s: s.split("\t")).countByKey()

defaultdict(int, {'ham': 4827, 'spam': 747})

### Задание 3. Считаем DF

Найдите 10 слов с самой высокой Document Frequency (количество смс, в которых слово встречается хотя бы раз). Должно получиться что-то такое:
```python
[('to', 1635),
 ('you', 1220),
 ('I', 1128),
 ('a', 1103),
 ('the', 967),
 ('in', 725),
 ('and', 704),
 ('is', 662),
 ('i', 592),
 ('for', 575)]
```

In [118]:
(
    src
    .map(lambda x: x.split('\t')[1])
    .map(lambda x: x.replace(r'[.?,]', ''))
    .map(lambda x: x.lower())
    .flatMap(lambda x: x.split(' '))
    .map(lambda x: (x, 1))
    .reduceByKey(lambda x,y: (x+y))
    .map(lambda x: (x[1], x[0]))
    .sortByKey(False)
    .take(10)
)

[(2237, 'to'),
 (2217, 'i'),
 (1921, 'you'),
 (1433, 'a'),
 (1328, 'the'),
 (998, 'u'),
 (968, 'and'),
 (869, 'is'),
 (859, 'in'),
 (755, 'my')]

не годится, счетчики не бьются. Пробуем другой подход, через Counter.

In [271]:
sms_lists = [text.split(" ") for text in text] # бьёт наши смс на слова
uniq_words = [set(sms_list) for sms_list in sms_lists] # оставляет только уникальные слова

counter = Counter() # заводим счетчик
for word in uniq_words:
    counter.update(word)
for word in counter.most_common(10):
    print(word)

('to', 1635)
('you', 1220)
('I', 1128)
('a', 1103)
('the', 967)
('in', 725)
('and', 704)
('is', 662)
('i', 592)
('for', 575)


## Задание 4. Нумеруем документы

Присвойте каждому документу уникальный идентификатор, целое число. Пример ответа:
```python
[(0,
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'),
 (1, 'Ok lar... Joking wif u oni...'),
 (2,
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"),
 (3, 'U dun say so early hor... U c already then say...'),
 (4, "Nah I don't think he goes to usf, he lives around here though")]
```

In [231]:
accum = sc.accumulator(0)
accum



# src.map(lambda s: s.split("\t")[1]).map(lambda word: (accum.add(x), word)).take(5)


In [272]:
# src.map(lambda s: s.split("\t")[1]).map(lambda word: (accum.add(1), word)).take(5)
# src.map(lambda s: s.split("\t")[1]).map(lambda word: (word, 1)).reduceByKey(lambda a, b: ((int(a)+1), b)).take(5)

# src.map(lambda s: s.split("\t")[1]).reduce(lambda word: (countFun(1), word)).take(5)
# src.map(lambda s: s.split("\t")[1]).map(lambda word: (1, word)).map(lambda word: (word[0]+1, word[1])).take(5)

Блин, не могу победить методами Spark эту историю. Тогда по старинке...

In [276]:
pd.DataFrame(all_text).head(5)

,0
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."


In [274]:
list(zip(pd.DataFrame(all_text).index, pd.DataFrame(all_text)[0]))[:5]

[(0,
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'),
 (1, 'Ok lar... Joking wif u oni...'),
 (2,
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"),
 (3, 'U dun say so early hor... U c already then say...'),
 (4, "Nah I don't think he goes to usf, he lives around here though")]

## Задание 5. Считаем IDF

Посчитайте для каждого слова его [IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
Примерный ответ:

```python
[('amore', 8.625868208041894),
 ('jurong', 8.625868208041894),
 ('Go', 5.986810878426635),
 ('got', 3.3477535488113763),
 ('there', 3.9530393735799874)]
```

In [277]:
from pyspark.mllib.feature import HashingTF, IDF

In [307]:
hashingTF = HashingTF()
tf = hashingTF.transform(text)
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [308]:
tfidf.take(5)

[SparseVector(1048576, {35920: 1.7494, 40200: 1.0154, 105642: 3.3644, 151034: 1.8668, 173606: 0.2036, 211440: 0.194, 238153: 0.6522, 244458: 2.2493, 265159: 0.5334, 279832: 0.6426, 296409: 0.5233, 335453: 0.8987, 348473: 2.6702, 372567: 1.3867, 388504: 1.468, 437751: 0.974, 469732: 2.2403, 584844: 0.6328, 599511: 2.1429, 702216: 0.805, 702740: 0.4601, 734443: 0.6716, 777769: 0.3253, 858685: 1.6321, 875351: 0.1334, 891534: 0.3373, 897367: 0.9769, 954647: 1.4384, 968035: 0.7957}),
 SparseVector(1048576, {35920: 0.4374, 40200: 0.1692, 105642: 2.5233, 186435: 1.4777, 238153: 0.0932, 243360: 1.7375, 265159: 0.1524, 335453: 0.4494, 372567: 0.6933, 388504: 0.2447, 646712: 2.9806, 702216: 0.4025, 875351: 0.0351, 897367: 0.2442, 968035: 0.2273}),
 SparseVector(1048576, {24138: 10.318, 26255: 3.8554, 27527: 9.1967, 40200: 1.0154, 40649: 5.0071, 105642: 0.4206, 151034: 3.7336, 173013: 2.9444, 178773: 2.4523, 186435: 1.4777, 211440: 1.1639, 238153: 0.4659, 265159: 0.4572, 279832: 1.6065, 292370: 8

Я не очень понимаю, как из этого векторизированного набора вернуть результат.... Может отказаться от хэша?

In [306]:
idf_new = IDF().fit(text)
# tfidf = idf_new.transform(tf)

22/02/11 18:44:37 ERROR Executor: Exception in task 0.0 in stage 260.0 (TID 318)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/jupyter/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/jupyter/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/jupyter/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/jupyter/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "/jupyter/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/mllib/linalg/__init__.py", line 83, in _convert_to_vector
    raise TypeError("Cannot convert type %s into Vector" % type(l))


Py4JJavaError: An error occurred while calling o5185.fitIDF.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 260.0 failed 1 times, most recent failure: Lost task 0.0 in stage 260.0 (TID 318, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/jupyter/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/jupyter/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/jupyter/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/jupyter/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "/jupyter/lib/python3.7/sit

Не годится. Ну ок, давайте другие методы пробовать...

In [321]:
from pyspark.sql import SparkSession

sp = SparkSession.builder.appName('nlp').getOrCreate()
data = sp.read.csv("SMSSpamCollection",inferSchema=True,sep='\t')
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

pd_text = data.toPandas()
pd_text.head(10)

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [322]:
# Сгруппируем по классу

pd_data.groupby('class').describe()

text                                                               
      count unique                                                top freq
class                                                                     
ham    4827   4518                             Sorry, I'll call later   30
spam    747    653  Please call our customer service representativ...    4

In [325]:
!pip install sklearn

     |████████████████████████████████| 24.8 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 38.1 MB 123 kB/s  eta 0:00:01    |████▎                           | 5.2 MB 42.4 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 53.8 MB/s eta 0:00:01
Using legacy setup.py install for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn ... done
You should consider upgrading via the '/jupyter/bin/python3.7 -m pip install --upgrade pip' command.


In [326]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(pd_data['text'])
idf = vectorizer.idf_
features = vectorizer.get_feature_names_out()
# features

In [327]:
idf

array([7.22815232, 6.29384309, 8.93290042, ..., 8.93290042, 8.93290042,
       8.93290042])

In [329]:
# vectorizer.get_feature_names()

In [337]:
idf_dict = dict(zip(vectorizer.get_feature_names(), idf))


# [('amore', 8.625868208041894),
#  ('jurong', 8.625868208041894),
#  ('Go', 5.986810878426635),
#  ('got', 3.3477535488113763),
#  ('there', 3.9530393735799874)]


slice_list = ['amore', 'jurong', 'go', 'got', 'there']

for word in slice_list:
    print(f'{word}, {idf_dict[word]}')
# idf_dict['amore']
# idf_dict

amore, 8.932900415768374
jurong, 8.932900415768374
go, 4.046317770342098
got, 4.187968287405124
there, 4.2554095682006565


Подсчитанные idf через sklearn не совпадает с тем, что даны в качестве ответа у вас :(

## Задание 6. Считаем TF

Посчитайте для каждого документа [TF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) всех слов входящих в него
Примерный ответ:

```python
[(0,
  [('Go', 0.05),
   ('until', 0.05),
   ('jurong', 0.05),
   ('point,', 0.05),
   ('crazy..', 0.05),
   ('Available', 0.05),
   ('only', 0.05),
   ('in', 0.05),
   ('bugis', 0.05),
   ('n', 0.05),
   ('great', 0.05),
   ('world', 0.05),
   ('la', 0.05),
   ('e', 0.05),
   ('buffet...', 0.05),
   ('Cine', 0.05),
   ('there', 0.05),
   ('got', 0.05),
   ('amore', 0.05),
   ('wat...', 0.05)]),
 (1,
  [('Ok', 0.16666666666666666),
   ('lar...', 0.16666666666666666),
   ('Joking', 0.16666666666666666),
   ('wif', 0.16666666666666666),
   ('u', 0.16666666666666666),
   ('oni...', 0.16666666666666666)]),
 (2,
  [('Free', 0.03571428571428571),
   ('entry', 0.07142857142857142),
   ('in', 0.03571428571428571),
   ('2', 0.03571428571428571),
   ('a', 0.03571428571428571),
   ('wkly', 0.03571428571428571),
   ('comp', 0.03571428571428571),
   ('to', 0.10714285714285714),
   ('win', 0.03571428571428571),
   ('FA', 0.07142857142857142),
   ('Cup', 0.03571428571428571),
   ('final', 0.03571428571428571),
   ('tkts', 0.03571428571428571),
   ('21st', 0.03571428571428571),
   ('May', 0.03571428571428571),
   ('2005.', 0.03571428571428571),
   ('Text', 0.03571428571428571),
   ('87121', 0.03571428571428571),
   ('receive', 0.03571428571428571),
   ('question(std', 0.03571428571428571),
   ('txt', 0.03571428571428571),
   ("rate)T&C's", 0.03571428571428571),
   ('apply', 0.03571428571428571),
   ("08452810075over18's", 0.03571428571428571)]),
 (3,
  [('U', 0.18181818181818182),
   ('dun', 0.09090909090909091),
   ('say', 0.09090909090909091),
   ('so', 0.09090909090909091),
   ('early', 0.09090909090909091),
   ('hor...', 0.09090909090909091),
   ('c', 0.09090909090909091),
   ('already', 0.09090909090909091),
   ('then', 0.09090909090909091),
   ('say...', 0.09090909090909091)]),
 (4,
  [('Nah', 0.07692307692307693),
   ('I', 0.07692307692307693),
   ("don't", 0.07692307692307693),
   ('think', 0.07692307692307693),
   ('he', 0.15384615384615385),
   ('goes', 0.07692307692307693),
   ('to', 0.07692307692307693),
   ('usf,', 0.07692307692307693),
   ('lives', 0.07692307692307693),
   ('around', 0.07692307692307693),
   ('here', 0.07692307692307693),
   ('though', 0.07692307692307693)])]
```

Короче, я уже полностью разуверился, что смогу хоть что-то делать методами Spark ((

In [342]:
def tf_counter(text):
    collection_text = collections.Counter(text)
    for word in collection_text:
        collection_text[word] = collection_text[word]/float(len(text))
    return collection_text

TF_list = []

for word in sms_lists:
    TF_list.append(tf_counter(word))

TF_list

[Counter({'Go': 0.05,
          'until': 0.05,
          'jurong': 0.05,
          'point,': 0.05,
          'crazy..': 0.05,
          'Available': 0.05,
          'only': 0.05,
          'in': 0.05,
          'bugis': 0.05,
          'n': 0.05,
          'great': 0.05,
          'world': 0.05,
          'la': 0.05,
          'e': 0.05,
          'buffet...': 0.05,
          'Cine': 0.05,
          'there': 0.05,
          'got': 0.05,
          'amore': 0.05,
          'wat...': 0.05}),
 Counter({'Ok': 0.16666666666666666,
          'lar...': 0.16666666666666666,
          'Joking': 0.16666666666666666,
          'wif': 0.16666666666666666,
          'u': 0.16666666666666666,
          'oni...': 0.16666666666666666}),
 Counter({'Free': 0.03571428571428571,
          'entry': 0.07142857142857142,
          'in': 0.03571428571428571,
          '2': 0.03571428571428571,
          'a': 0.03571428571428571,
          'wkly': 0.03571428571428571,
          'comp': 0.03571428571428571,
    

А тут вроде даже похоже получилось...

## Задание 7. Считаем TF-IDF

Посчитайте для каждого документа [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) всех слов входящих в него
Примерный ответ:

```python
[(0,
  [('jurong', 0.4312934104020947),
   ('point,', 0.4312934104020947),
   ('Cine', 0.4312934104020947),
   ('crazy..', 0.4312934104020947),
   ('buffet...', 0.4312934104020947),
   ('amore', 0.4312934104020947),
   ('la', 0.3966360513740974),
   ('Available', 0.3763627959686892),
   ('bugis', 0.36197869234610014),
   ('wat...', 0.31139864676217616),
   ('Go', 0.29934054392133175),
   ('world', 0.2896327431992839),
   ('until', 0.26838858350102063),
   ('e', 0.22257404690731286),
   ('great', 0.2188686482996267),
   ('there', 0.19765196867899937),
   ('n', 0.19404680398393218),
   ('only', 0.18350205752203164),
   ('got', 0.16738767744056882),
   ('in', 0.10198482765936095)]),
 (1,
  [('Joking', 1.4376447013403155),
   ('oni...', 1.3221201712469912),
   ('lar...', 0.9654424773309462),
   ('wif', 0.9011653971956154),
   ('Ok', 0.6751928715897517),
   ('u', 0.3963042222555774)]),
 (2,
  [('FA', 0.5666229305344248),
   ('entry', 0.40215256674913585),
   ("08452810075over18's", 0.2833114652672124),
   ('question(std', 0.2833114652672124),
   ("rate)T&C's", 0.2833114652672124),
   ('21st', 0.2688305685490637),
   ('2005.', 0.2688305685490637),
   ('Cup', 0.2688305685490637),
   ('tkts', 0.25855620881864294),
   ('87121', 0.25855620881864294),
   ('wkly', 0.22583153982313742),
   ('comp', 0.22583153982313742),
   ('final', 0.216461387520727),
   ('apply', 0.21135064310498866),
   ('May', 0.20688053085663133),
   ('receive', 0.1842904394729345),
   ('win', 0.18008390248520656),
   ('Free', 0.17722523435400883),
   ('Text', 0.17457852106280447),
   ('txt', 0.15483602738905367),
   ('to', 0.13140751336184356),
   ('2', 0.09667765549174095),
   ('in', 0.0728463054709721),
   ('a', 0.05785997102815683)]),
 (3,
  [('hor...', 0.7211564570438135),
   ('say...', 0.684295992670344),
   ('U', 0.5608372114333577),
   ('early', 0.49912490837388584),
   ('dun', 0.44015259764760295),
   ('c', 0.44015259764760295),
   ('already', 0.43810961075196125),
   ('say', 0.3979429969084122),
   ('then', 0.3336401045855121),
   ('so', 0.2712691942821618)]),
 (4,
  [('he', 0.6185689264698156),
   ('usf,', 0.6102093098063037),
   ('lives', 0.6102093098063037),
   ('Nah', 0.5138429276143524),
   ('though', 0.455216769764591),
   ('goes', 0.415922491013361),
   ('around', 0.3641575315331744),
   ('here', 0.3324463934490557),
   ("don't", 0.31162747919527006),
   ('think', 0.30125676974842763),
   ('I', 0.12289744430645304),
   ('to', 0.0943438557469646)])]
```

In [349]:
def ifidf_counter(text):
    tfidf_db = vectorizer.transform([text]).todense()
    feature_index = tfidf_db[0,:].nonzero()[1]
    tfidf_scores = zip([features[word] for word in feature_index], [tfidf_db[0, ind] for ind in feature_index])
    return dict(tfidf_scores)

TFIDF_list = []
for word in pd_data['text']:
    TFIDF_list.append(ifidf_counter(word))

TFIDF_list[:5]

[{'amore': 0.326467645747019,
  'available': 0.2441904570208067,
  'buffet': 0.31164925159555484,
  'bugis': 0.2758032215037706,
  'cine': 0.2758032215037706,
  'crazy': 0.25282967656470956,
  'go': 0.14787938686701288,
  'got': 0.15305623970002818,
  'great': 0.1803467768751424,
  'in': 0.10700055412297083,
  'jurong': 0.326467645747019,
  'la': 0.2758032215037706,
  'only': 0.15603453765460987,
  'point': 0.25535113536077003,
  'there': 0.15552099304359127,
  'until': 0.23001892323914586,
  'wat': 0.182416008162686,
  'world': 0.22083422107921832},
 {'joking': 0.5236769406481623,
  'lar': 0.4083285209202484,
  'ok': 0.2719043567370419,
  'oni': 0.5466195966483365,
  'wif': 0.43163099770972085},
 {'08452810075over18': 0.2303218201445323,
  '2005': 0.2225516693126023,
  '21st': 0.2225516693126023,
  '87121': 0.21652467209269544,
  'apply': 0.16555585196993128,
  'comp': 0.1907167794206798,
  'cup': 0.19780310875099452,
  'entry': 0.35301452895940433,
  'fa': 0.4606436402890646,
  'fina